In [4]:
#import necessary package and collect initial variables
import openmdao.api as om


In [2]:
#define discipline group and call analyses 
class aero(om.Group):
    def setup(self):
        
        pass

In [3]:
#define analyses, this can be done in different files if wanted
class analysis1(om.ExplicitComponent):
    """
    Sample analysis definition, with inputs & outputs 
    """

    def setup(self):

        pass
    
    def setup_partials(self):
        pass

    def compute(self, inputs, outputs):
        pass